In [1]:
import tensorflow as tf
import numpy as np
import gym
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
env = gym.make('CartPole-v0')

[2017-09-10 19:54:43,821] Making new env: CartPole-v0


In [80]:
print(env.observation_space.shape)
print(env.observation_space.sample())
print(env.action_space.n)
print(env.action_space.sample())

(4,)
[ -2.18249957e+00  -1.52003326e+37   2.61521863e-01  -1.36268295e+37]
2
1


In [86]:
class Qlearner():
    def __init__(self, epsilon, alpha, gamma):
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.model = Sequential()
        self.model.add(Dense(32, input_dim=4, activation='relu'))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(2, activation='linear'))
        self.model.compile(loss='mse', optimizer = 'adam')
    
    def get_qvalue(self, state):
        return self.model.predict(state)[0]
    
    def get_action(self, state):
        if self.epsilon > np.random.rand():
            return int(np.random.randint(low=0, high=2, size=1))
        else:
            return np.argmax(self.model.predict(state))
    
    def update_model(self, state, next_state, reward, action):
        q_curr = self.get_qvalue(state)
        q_ref = reward + self.gamma * np.amax((self.get_qvalue(next_state)))
        q_curr[action] = q_curr[action] + self.alpha * (q_ref - q_curr[action])
        self.model.fit(x=state, y=q_curr.reshape(1, 2), epochs=1, verbose=0)

In [87]:
agent = Qlearner(epsilon=1, alpha=0.1, gamma=0.9)
episodes = 100

In [88]:
s = env.reset()
print(agent.get_qvalue(s.reshape((1,4))))
print(agent.epsilon)

[ 0.00631537 -0.00217974]
1


In [89]:
def generate_sessions(t_max = 1000):
    total_reward = 0
    s = env.reset()
    
    for t in range(t_max):
        s = s.reshape((1, 4))
        agent.get_qvalue(s)
        a = agent.get_action(s)
        
        next_s, r, done, _ = env.step(a)
        
        agent.update_model(action=a, next_state=next_s.reshape((1, 4)), state=s, reward=r)
        
        s = next_s
        total_reward = total_reward + r
        if done:
            break
    return total_reward

In [90]:
for i in range(episodes):
    rewards = [generate_sessions() for i in range(100)]
    if agent.epsilon > 0.01:
        agent.epsilon *= 0.95
    print ("episode {0} mean reward:{1}\tepsilon:{2}".format(i+1, np.mean(rewards),agent.epsilon))

episode 1 mean reward:24.02	epsilon:0.95
episode 2 mean reward:22.72	epsilon:0.9025
episode 3 mean reward:23.39	epsilon:0.8573749999999999
episode 4 mean reward:20.12	epsilon:0.8145062499999999
episode 5 mean reward:19.81	epsilon:0.7737809374999999
episode 6 mean reward:19.77	epsilon:0.7350918906249998
episode 7 mean reward:18.24	epsilon:0.6983372960937497
episode 8 mean reward:19.75	epsilon:0.6634204312890623
episode 9 mean reward:18.44	epsilon:0.6302494097246091
episode 10 mean reward:14.17	epsilon:0.5987369392383786
episode 11 mean reward:16.68	epsilon:0.5688000922764596
episode 12 mean reward:13.48	epsilon:0.5403600876626365
episode 13 mean reward:14.03	epsilon:0.5133420832795047
episode 14 mean reward:14.36	epsilon:0.48767497911552943
episode 15 mean reward:12.93	epsilon:0.46329123015975293
episode 16 mean reward:12.97	epsilon:0.44012666865176525
episode 17 mean reward:13.18	epsilon:0.41812033521917696
episode 18 mean reward:13.53	epsilon:0.3972143184582181
episode 19 mean reward: